## **DS-GA 1007 $\:$ | $\,$ Lecture 10**

## **Programming for Data Science**

<br>

---

#### Jeremy Curuksu, PhD

#### NYU Center for Data Science

#### jeremy.cur@nyu.edu

#### November 28, 2022


## **Pandas: Advanced Data Objects (Part 4)**

<br>

### **Last Lecture**:

### ▶ Statistical Analysis with Group Operations in Pandas 

### (`groupby`, `aggregate`, `filter`, `transform`, `apply`)

### ▶  Merging Data Frames in Pandas

<br>

### **Today**:

### ▶ Manipulating time series with Pandas

### ▶ Case studies analyzing and vizualizing time series ( `resample`, `shift`, `rolling`, `groupby` )

### ▶ Extra time for class survey

# Topics covered on Pandas in this course

• **Part 1**: Pandas series and data frames, indexing and selection, fancy indexing, Boolean and hierarchical indexing, reshaping

• **Part 2**: Operating on data with Pandas, handling missing values

• **Part 3**: Statistical analysis with Pandas (aggregation, group operations, merging, joining)

• **Part 4**: Manipulating, analyzing and visualizing time series with Pandas



# Introduction to time series in Pandas

Most of the material in this lecture is based on contents from the book "Python Data Science Handbook" by Jake VanderPlas: https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

Pandas was originally developed in the context of financial modeling and has an extensive set of tools for working with dates, times, and time-indexed data.


In Pandas, date and time can be either:


- **Time stamps** reference moments in time e.g., *Monday 2022-11-28 7.00PM EST*



- **Time intervals** and **periods** reference lengths of time between a beginning and end point e.g., *the year 2021, a 24h period*



- **Time durations** or *time deltas* reference exact lengths of time e.g., *30 seconds*





# Import Pandas as a library:


In [ ]:
import pandas as pd # Abbreviations "pd" is semi-standardized...
import numpy as np  # We will be using NumPy occasionally in this lecture

# 1) Defining Dates and Times in Python


## Python ``datetime`` object

**Create a `datetime` object with the `datetime` library**

We can manually create a date using the ``datetime`` function:

In [ ]:
from datetime import datetime

date = datetime(year = 2022, month = 11, day = 28)
date

In [ ]:
date2 = datetime(year = 2022, month = 12, day = 5)

# Python automatically converts between date vs. time objects
time = date2 - date
time

**Convert a string into a `datetime` object with the `dateutil` library**

We can parse dates from different string formats using the ``dateutil`` function:

In [ ]:
from dateutil import parser

date = parser.parse("28th of November 2022")
date

**Access attributes of a `datetime` object with the `strftime()` method**

We can access attributes of ``datetime`` objects, such as the day of the week, using the ``strftime()`` method:

In [ ]:
date.strftime('%A')


The standard string format codes for printing dates (such as ``"%A"``) can be found in the [strftime section](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) of the Python's [datetime documentation](https://docs.python.org/3/library/datetime.html)
(also check out [dateutil's online documentation](http://labix.org/python-dateutil), and the package [pytz](http://pytz.sourceforge.net/) for working with time zones).

## NumPy arrays of times: ``datetime64``

Python ``datetime`` and ``dateutil`` are flexible and easy to use, but do not scale well to large data sets, for the same reasons that *list* objects do not scale well to large data sets.

**The ``datetime64`` NumPy object**

NumPy offers the ``datetime64`` data type which encodes dates as 64-bit integers, and allows vectorized operations on homogeneous arrays of dates represented compactly:

In [ ]:
import numpy as np

date = np.array('2022-11-28', dtype=np.datetime64)
date

Operators applied to `datetime64` objects recast integers into `datetime64` objects:

In [ ]:
date + 10

**Create an array of `datetime64` objects with the `arange()` NumPy method**

In [ ]:
np.arange(date, date + 10, dtype = np.datetime64) 

In [ ]:
# Or more directly... 
date + np.arange(10) # Vectorized operation on NumPy date objects

**Trade-off between time resolution and maximum time span**

``datetime64`` and ``timedelta64`` objects are built on a fundamental time unit and limited to 64-bit precision, meaning the range of encodable times is 264 times this fundamental unit. Thus, ``datetime64`` imposes a trade-off between time resolution and maximum time span.

In [ ]:
np.datetime64('2022-11-28') # Day resolution

In [ ]:
np.datetime64('2022-11-28 18:55') # Minute resolution

In [ ]:
np.datetime64('2022-11-28 18:55:10.50','ns') # Nanosecond resolution

Below are available `datetime64` format codes defining the resolution of the fundamental time unit and its associated time span:

|Code    | Meaning     | Time span (relative) | Time span (absolute)   |
|--------|-------------|----------------------|------------------------|
| ``Y``  | Year	       | ± 9.2e18 years       | [9.2e18 BC, 9.2e18 AD] |
| ``M``  | Month       | ± 7.6e17 years       | [7.6e17 BC, 7.6e17 AD] |
| ``W``  | Week	       | ± 1.7e17 years       | [1.7e17 BC, 1.7e17 AD] |
| ``D``  | Day         | ± 2.5e16 years       | [2.5e16 BC, 2.5e16 AD] |
| ``h``  | Hour        | ± 1.0e15 years       | [1.0e15 BC, 1.0e15 AD] |
| ``m``  | Minute      | ± 1.7e13 years       | [1.7e13 BC, 1.7e13 AD] |
| ``s``  | Second      | ± 2.9e12 years       | [ 2.9e9 BC, 2.9e9 AD]  |
| ``ms`` | Millisecond | ± 2.9e9 years        | [ 2.9e6 BC, 2.9e6 AD]  |
| ``us`` | Microsecond | ± 2.9e6 years        | [290301 BC, 294241 AD] |
| ``ns`` | Nanosecond  | ± 292 years          | [ 1678 AD, 2262 AD]    |
| ``ps`` | Picosecond  | ± 106 days           | [ 1969 AD, 1970 AD]    |
| ``fs`` | Femtosecond | ± 2.6 hours          | [ 1969 AD, 1970 AD]    |
| ``as`` | Attosecond  | ± 9.2 seconds        | [ 1969 AD, 1970 AD]    |

Note the time zone is automatically set to the local time on the computer executing the code.


# 2) Manipulating Time Series with Pandas


## Pandas offers three types of date and time objects 

1. ``Timestamp`` with associated index structure ``DatetimeIndex`` 

2. ``Period`` with associated index structure ``PeriodIndex``

3. ``Timedelta`` with associated index structure ``TimedeltaIndex``


## The ``Timestamp`` Pandas object

* Pandas offers the ``Timestamp`` object, which combines the ease-of-use of ``datetime`` and ``dateutil`` with the efficient storage and vectorized interface of ``datetime64``


* The `DatetimeIndex` object is used to index Pandas data frames by `Timestamp`. Pandas constructs a ``DatetimeIndex`` whenever it is given an array of ``Timestamp`` objects


### Create a `Timestamp` object using the `to_datetime` Pandas method

In [ ]:
date = pd.to_datetime("November 28th 2022")
date 

In [ ]:
dates = pd.to_datetime([datetime(2022, 11, 28), '29th of November, 2022',
                       '2022-Nov-30', '12-01-2022', '20221202'])
dates

The `strftime()` method can be used to access attributes of `Timestamp` objects (usage identical to usage for `datetime` objects)

In [ ]:
date.strftime('%A')

### Pandas automatically converts between date and time objects

In [ ]:
dates - dates[0]

## The `DatetimeIndex` Pandas object 

* The `DatetimeIndex` object is used to index Pandas data frames by `Timestamp`.


* Pandas constructs a ``DatetimeIndex`` whenever it is given an array of ``Timestamp`` objects.

### Create a ``DatetimeIndex`` object using the `DatetimeIndex()` Pandas method

In [ ]:
index = pd.DatetimeIndex(['2022-11-28', 
                          '2022-11-29',
                          '2022-11-30', 
                          '2022-12-01',
                          '2022-12-02',
                          '2022-12-03',
                          '2022-12-04',
                          '2022-12-05',
                          '2022-12-06',
                          '2022-12-07'
                         ])

data = pd.Series(range(100, 100 + len(index)), index = index)
data

### Create a ``DatetimeIndex`` object from an array of time objects itself created using `arange()` combined with `to_timedelta`

We can't use `arange()` directly because recasting of integers (as we saw with NumPy) is no longer supported in Pandas. Instead we can use the `to_timedelta` Pandas method to recast an array of integers created with `arange()` into an array of `timedelta` objects.

Operators applied to `Timestamp` objects recast `timedelta` objects into `Timestamp` objects.

In [ ]:
date = pd.to_datetime("November 28st 2022")

index = date + pd.to_timedelta(np.arange(10), 'D')
index

In [ ]:
# Again let's use this DatetimeIndex object to index a data frame
data = pd.Series(range(100, 100 + len(index)), index = index)
data

### Create a ``DatetimeIndex`` object using the `date_range()` Pandas method

Pandas offers the method ``date_range()`` to create regular sequences of dates, which accepts a start date, an end date, and an optional frequency code. By default, the frequency is one day:

In [ ]:
pd.date_range('2022-11-28', '2022-12-31')

The date range can also be specified with a start date and a number of periods:


In [ ]:
pd.date_range('2022-11-28', periods=10)

The length of each period can be modified using the `freq` argument, which defaults to D (days). For example, here is a range of 10 periods of 4 hours each, starting today:

In [ ]:
pd.date_range('2022-11-28', periods=10, freq='4H')

As another example, here is a sequence of *durations* increasing by an hour:

In [ ]:
pd.timedelta_range(0, periods=10, freq='H')

### Pandas Frequencies 

The table below summarizes the main codes available to specify a time frequency in Pandas:

| Code   | Description         | Code   | Description          |
|--------|---------------------|--------|----------------------|
| ``D``  | Calendar day        | ``B``  | Business day         |
| ``W``  | Weekly              |        |                      |
| ``M``  | Month end           | ``BM`` | Business month end   |
| ``Q``  | Quarter end         | ``BQ`` | Business quarter end |
| ``A``  | Year end            | ``BA`` | Business year end    |
| ``H``  | Hours               | ``BH`` | Business hours       |
| ``T``  | Minutes             |        |                      |
| ``S``  | Seconds             |        |                      |
| ``L``  | Milliseconds         |        |                      |
| ``U``  | Microseconds        |        |                      |
| ``N``  | nanoseconds         |        |                      |

### Indexing of data frames based on `DatetimeIndex`

In [ ]:
# Basic operation on data indexed by time stamps:
data - data[0] 

In [ ]:
# Slicing of data indexed by time stamps:
data['2022-11-28':'2022-12-01'] 

In [ ]:
# Temporal order matters:
data['2022-11-30':'2022-11-28'] 

In [ ]:
# Semantically a new type of hierarchical indexing:
data['2022']

# 3) Analyzing and Visualizing Time Series

Let's apply the above on case studies, and introduce some time series analysis methods: 

* **Resampling** time series


* **Shifting** time series


* **Rolling Windows** on time series

## Use Case 1: Financial Stock Trading 

In [ ]:
from pandas_datareader import data

amzn = data.DataReader('AMZN', start = '2010', end = '2023', 
                       data_source = 'yahoo') 

amzn.tail()

In [ ]:
from matplotlib import pyplot as plt
amazoncolor = [1, 0.6, 0]

amzn = amzn['Close'] # Let's focus on closing price for simplicity

amzn.plot(color = amazoncolor);

### Resampling time series

Time series can be resampled at different frequencies, for example to report the average of each year using the `resample` method, or the value *at the end* of each year using the `asfreq` method:

In [ ]:
amzn.plot(alpha=0.5, style = '-')

amzn.resample('BA').mean().plot(style = ':')

amzn.asfreq('BA').plot(style = '--')

plt.legend(['input', 'mean/y', 'EoY'], loc = 'upper left');

### Shifting time series

Time series can be shifted by a preset time,  using the method `shift()` to shift the data against the original indices, or using the method `tshift()` to shift the indices. In both cases, the shift is specified in multiples of a frequency. **Positive shifts move the data forward in time, negative shifts move the data backward in time**.

Here is an example using `shift()` that shifts the Amazon stock price by 365 days backward in time, and divides the shifted time series by the original time series to compute the 1-year return on investment for the Amazon stock over the timeline of the dataset:

In [ ]:
# Apply a frequency to the data
amzn = amzn.asfreq('D', method = 'pad')

# Compute the 1-year ROI
ROI = 100 * (amzn.shift(-365) / amzn - 1)

# Plot the 1-year ROI against time line of the dataset
ROI.plot(color = amazoncolor)
plt.ylabel('% 1-year ROI');

### Rolling Windows on time series

Time series can be aggregated and summarized using any of the `groupby` methods seen in the previous lecture (which we will illustrate in the next case study).

This operation is so common when analyzing and vizualizing time series (to *smooth* potentially noisy time series), that Pandas offers the method `rolling` to quickly create rolling windows on time series, and adds custom options (e.g., centering aggregate values on each window).

Here is an example to vizualize the one-year centered rolling mean and standard deviation of the Amazon stock prices:

In [ ]:
rolling = amzn.rolling(365, center = True)

ts = pd.DataFrame({'input': amzn,
                     'one-year rolling mean': rolling.mean(),
                     'one-year rolling std dev': rolling.std()})

ax = ts.plot(style = ['-', '--', ':'])
ax.lines[0].set_alpha(0.3)

Note the subtle difference between using `resample` and `rolling` even for the same statistic e.g., mean per year: The resampling approach computes a mean per year every year, while the rolling window approach computes a mean per year every day.

As with `groupby` operations, the `aggregate()` and `apply()` methods can be used for custom rolling computations. We will look at example of `groupby` operations on time series in the next case study (see below).



### For more analyses specifically on *financial stock market* data:

There are excellent open-source packages available that can accelerate data analysis and modeling for specific types of datasets. For financial stock market datasets, the library `ta` can be used to quickly compute standard financial indicators

In [ ]:
# !pip install ta # "Technical Analysis" of financial time series datasets
from ta import add_all_ta_features

amzn = data.DataReader('AMZN', start = '2010', end = '2023', 
                       data_source = 'yahoo') 

financial_ind = add_all_ta_features(amzn, open='Open', high='High', low='Low', close='Close', volume='Volume')
financial_ind.shape

In [ ]:
financial_ind

## Use Case 2: Seattle Bike Riding

The Fremont Bridge Bicycle Counter began operation in October 2012 and **records the number of bikes that cross the bridge using the bicycle pathways**. Inductive loops on the east and west pathways count the passing of bicycles regardless of travel direction. The data consists of a date/time column: `Date`, an east pathway count: `Fremont Bridge East Sidewalk`, a west pathway count: `Fremont Bridge West Sidewalk`, and the total count of both sidewalks: `Fremont Bridge Total`. 

**The count variables represent the total number of bicycles detected during 1-hour periods specified in the column `Date`**. Direction of travel is not specified, but in general most traffic on the Fremont Bridge East sidewalk is travelling northbound and most traffic on the Fremont Bridge West sidewalk is travelling southbound.


https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD

In [ ]:
# This command will download the data in bash (from a Linux terminal)
# !curl -o FremontBridge.csv https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD

In [ ]:
data = pd.read_csv('FremontBridge.csv', index_col='Date', parse_dates=True)

In [ ]:
data.shape

In [ ]:
data.head(10)

In [ ]:
# Rename columns for simplicity
data = data.rename(columns={"Fremont Bridge Total": "Total", 
                     "Fremont Bridge East Sidewalk": "East", 
                     "Fremont Bridge West Sidewalk": "West"}
                  )
data.head(10)

In [ ]:
data.dropna().describe()

In [ ]:
%matplotlib inline

import seaborn
from matplotlib import pyplot as plt

data.plot(figsize = (5, 5))
plt.ylabel('Hourly Bicycle Count')

**Observed overall trend**: The impact of the Covid-19 pandemic was very significant...

In [ ]:
data['East'].plot(figsize = (5, 5))
plt.ylabel('Hourly Bicycle Count: East')

In [ ]:
data['West'].plot(figsize = (5, 5))
plt.ylabel('Hourly Bicycle Count: West')

### Resampling time series

In [ ]:
data.resample('W').sum() # Resampled to report the sum of counts per week

In [ ]:
weekly = data.resample('W').sum()
weekly.plot(style = [':', '--', '-'])
plt.ylabel('Weekly bicycle count');

**Observed seasonal trends**:  Peak in the summer, and additional week-to-week fluctuations. People bicycle more in the summer than in the winter, and even within a particular season the bicycle use varies from week to week, likely dependent on weather.

### Aggregation and `groupby` operations on time series


#### Aggregating data per hour of the day (using the `time` method)

The `time` attribute  of a `DatetimeIndex` object is a NumPy array of `datetime.time` objects corresponding to the *time* part (independent of any particular *date*) of each element in the `DatetimeIndex` object.

In this case study the length of each time period is 1 hour.

In [ ]:
data.index # The index contains 145814 entries

In [ ]:
data.index.time # This NumPy arrays also contains 145814 elements

In [ ]:
data.groupby(data.index.time).mean()

In [ ]:
hourly_ticks = 4 * 60 * 60 * np.arange(6) # Hourly ticks for x-axis

avg_by_time = data.groupby(data.index.time).mean()

avg_by_time.plot(xticks = hourly_ticks, style = [':', '--', '-']);

**Observed hourly trends**:  The hourly traffic reveals a bimodal distribution with maxima around 8am and 5pm.

#### Aggregating data per day of the week (using the `dayofweek` method)

Similarly, the `dayofweek` attribute  of a `DatetimeIndex` object is a NumPy array of integers corresponding to the day of the week (independent of any particular time and any particular week, month, or year) of each element in the `DatetimeIndex` object.

`dayofweek` assumes the week starts on Monday which is noted by 0, and ends on Sunday which is noted by 6. 

In [ ]:
data.index.dayofweek # This NumPy arrays contains 145814 elements

In [ ]:
avg_by_weekday = data.groupby(data.index.dayofweek).mean()
avg_by_weekday.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
avg_by_weekday.plot(style=[':', '--', '-']);

**Observed weekly trends**:  The weekly traffic reveals a drop in traffic over the weekend

## Practice Exercice

We will add weather data downloaded from the National Climatic Data Center (`SeattleWeather.csv`) to the Fremont Bridge Bicycle case study. The data includes daily maximum and minimum temperatures (in F) and amounts of recorded rainfall at the Seattle airport (in inches).

Let's analyze the weather data in a pre-covid era (2012-2014) because the Covid-19 pandemic has had a very disruptive impact on bike riding.

In [ ]:
# Read the Seattle weather file
weather = pd.read_csv('SeattleWeather.csv', index_col='DATE', parse_dates=True)
weather.head()

In [ ]:
print(weather.mean())

In [ ]:
# 1) Plot the min and max temperature (in F) per week on a same canvas 

In [ ]:
weather['TMIN'].resample('W').min().plot()
weather['TMAX'].resample('W').max().plot()
plt.ylabel('Weekly Temperature Extremes (F)');

In [ ]:
# 2) Plot the total volume of precipitation (in inches) per week 

In [ ]:
weather['PRCP'].resample('W').sum().plot()
plt.ylabel('Weekly precipitation (in)');

In [ ]:
# 3) Count the total number of bike rides on the Fremont Bridge per day,
#    then merge these counts with the weather data on dates common to both datasets

In [ ]:
# Resample
daily_count = data.resample('D').sum()

# Merge
daily_all = pd.merge(daily_count, weather, left_index = True, right_index = True) # inner-join by default
daily_all

In [ ]:
# 4) Compute a linear regression model of total bike rides based on weather data, 
#    and plot model vs. actual for daily, weekly and monthly aggregates

In [ ]:
# Compute the linear regression model
from sklearn.linear_model import LinearRegression
x = daily_all[['TMIN', 'TMAX', 'PRCP']]
y = daily_all['Total']
clf = LinearRegression().fit(x, y)
daily_all['Weather model'] = clf.predict(x)

# Plot weather model vs actual 

# Daily aggregates
daily_all[['Total', 'Weather model']].plot()
plt.ylabel('Daily traffic');

# Weekly aggregates
daily_all[['Total', 'Weather model']].resample('W').sum().plot()
plt.ylabel('Weekly traffic');

# Monthly aggregates
daily_all[['Total', 'Weather model']].resample('M').sum().plot()
plt.ylabel('Monthly traffic');

## Thank you Everyone!